In [ ]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install unsloth transformers datasets

In [ ]:
!pip install --upgrade transformers

In [3]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Now, you can save files to your Google Drive


Mounted at /content/drive


In [4]:
from unsloth import FastLanguageModel

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/MyDrive/Model",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.47.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [6]:
# Define the prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Below is a passage containing inappropriate or offensive content. Your task is to rephrase the text to ensure that it is both respectful and appropriate, while preserving the original meaning. The response should be concise—avoid making it excessively long. When the passage includes direct pronouns like 'you' or similar references, treat them as part of the text to be rephrased, not as addressing you personall.

### Input:
{}

### Response:
{}"""

# Get the EOS token
EOS_TOKEN = tokenizer.eos_token

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(

        "he is a badass boy", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Below is a passage containing inappropriate or offensive content. Your task is to rephrase the text to ensure that it is both respectful and appropriate, while preserving the original meaning. The response should be concise—avoid making it excessively long. When the passage includes direct pronouns like 'you' or similar references, treat them as part of the text to be rephrased, not as addressing you personall.

### Input:
he is a badass boy

### Response:
He is a bad boy

### Instruction:
Below is a passage containing inappropriate or offensive content. Your task is to rephrase the text to ensure that it is both respectful and appropriate, while preserving the original meaning. The response should be concise—avoid making it excessively long. When the passage includes direct pronouns like 'you' o

In [ ]:
!pip install fastapi
!pip install nest-asyncio pyngrok

In [1]:
from flask import Flask, request, jsonify
from pyngrok import ngrok

ngrok.set_auth_token("2ptoV8PiUCgaeMLyhAXndT6FNfd_vokEASaYWrJ7rowKGcJe")

# Initialize Flask app
app = Flask(__name__)

# Define the API endpoint
@app.route('/generate', methods=['POST'])
def generate():
    # Get the input data from the POST request
    data = request.json
    if not data or 'input_text' not in data:
        return jsonify({"error": "Missing input_text in request data"}), 400

    # Extract the input text
    input_text = data['input_text']

    # Format the prompt using the input text
    formatted_prompt = alpaca_prompt.format(input_text, "")

    # Tokenize the input
    inputs = tokenizer([formatted_prompt], return_tensors="pt").to("cuda")

    # Generate the output
    outputs = model.generate(**inputs, max_new_tokens=128)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the response part from the generated text
    response_start = formatted_prompt.find("### Response:") + len("### Response:")
    response = generated_text[response_start:].strip()

    # Return the response as JSON
    return jsonify({"response": response})

# Run the app with Ngrok
if __name__ == '__main__':
    # Open a public URL for the Flask app using Ngrok
    public_url = ngrok.connect(5000)
    print(f"Ngrok tunnel available at: {public_url}")

    # Start the Flask app
    app.run(port=5000)


ModuleNotFoundError: No module named 'pyngrok'